# ユーザーの属するグループの確認

ユーザーが属するグループを確認します。JupyterHub上にユーザーが作成済みである必要があります。

グループは、所属グループに応じたリソース制限設定に使用します。

# グループの定義

グループを調べるユーザーのメールアドレス

In [ ]:
import re
user_emails = user_email_list.splitlines()
user_emails = [x.strip() for x in user_emails 
               if re.sub(r'#.*$', '', x).strip()]
user_emails

In [ ]:
import hashlib

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except for alphabets and digits
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h;
    return result;

user_names = [get_username_from_mail_address(x) for x in user_emails]
user_names

# グループの確認

[API Tokens](/hub/token)のページで、tokenを作成して以下に入力してください。

In [ ]:
import getpass
apitoken = getpass.getpass('JupyterHub API token')

In [ ]:
import os
api_url = os.environ['JUPYTERHUB_API_URL']

グループの作成（必要な場合）

In [ ]:
headers = {
    'Authorization': 'token %s' % apitoken,
}

In [ ]:
import requests
import json

user2mail = dict(zip(user_names, user_emails))

for username in user_names:
    r = requests.get(api_url + '/users/{}'.format(username),
                 headers=headers)
    assert r.status_code == 200
    user = r.json()
    print('{}({}){}: {}'.format(user2mail[username], username, 
                                ' [admin]' if user['admin'] else '',
                                ', '.join(user['groups'])))